# Text to text DPO

在选择模型的数据集的过程中：首先尝试了 encoder-decoder 架构的 t5-small ，但是模型的生成效果不好。于是改用作业中 work 的 Qwen2.5-0.5B-Instruct。尝试了 Dahoas/rm-static，Anthropic/hh-rlhf 数据集，但是他们都是多轮对话，同时 string 长度较大，针对 Human 的提问手动设置 label 并不容易。因此我们选择：**Intel/orca_dpo_pairs** 数据集。其中有

此外，我们使用了 **Qwen-2.5-0.5B-Instruct** 模型

由于使用 NPU，在命令行中输入 ``source /usr/local/Ascend/ascend-toolkit/set_env.sh``
运行 python 文件可行。但是使用 ipynb 文件就遇到了许多困难，环境变量不会“回传”给 Python 内核。
所以对于我使用的的服务器（910B）：尝试 **启动前先 source，再 jupyter notebook** 。
步骤如下：
```bash
conda activate align-anything
source /usr/local/Ascend/ascend-toolkit/set_env.sh
jupyter notebook --allow-root
```


### 准备工作


In [1]:
import os
# 设置 Hugging Face 镜像
os.environ["HF_HUB_ENDPOINT"] = "https://hf-mirror.com"
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
# os.environ["ASCEND_HOME_PATH"]="/usr/local/Ascend"

import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device=torch.device("npu:0")
print("my device is:",device)

/home/miniconda3/envs/align-anything/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/miniconda3/envs/align-anything/lib/python3.11/site-packages/torch_npu/__init__.py:248: UserWarning: On the interactive interface, the value of TASK_QUEUE_ENABLE is set to 0 by default.                      Do not set it to 1 to prevent some unknown errors
  warnings.warn("On the interactive interface, the value of TASK_QUEUE_ENABLE is set to 0 by default. \


my device is: npu:0


### 设置模型、数据集和超参数

在实现的过程中训练很容易不稳定，导致模型输入无关的 token 或 **不输出**，经过一系列的修改和调参，得到了以下的参数组合。

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
ref_model_name = "Qwen/Qwen2.5-0.5B-Instruct"  # 参考策略
# dataset_name="Dahoas/rm-static"
dataset_name="Intel/orca_dpo_pairs" # 采用单轮对话
save_dir = "./qwen-ft"
max_length=512

batch_size = 8
learning_rate = 8e-5
num_epochs = 1
beta = 0.1  # DPO 温度参数

### 加载模型

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device) # 显存占用 60 G
ref_model = AutoModelForCausalLM.from_pretrained(ref_model_name).eval().to(device)
model.config.sliding_window = None  # 显式禁用
ref_model.config.sliding_window = None  # 显式禁用
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

[2025-05-09 23:15:12,003] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to npu (auto detect)


/home/miniconda3/envs/align-anything/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


### 加载数据集

In [4]:
class PreferenceDataset(Dataset):
    def __init__(self, tokenizer, split="train", max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

        self.dataset = load_dataset(dataset_name, split=split)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        sample = self.dataset[idx]
        system_msg = sample["system"]
        user_msg = sample["question"]
        better_response = sample["chosen"]
        worse_response = sample["rejected"]

        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ]

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        text_tokens = self.tokenizer(text, add_special_tokens=False)
        text_len = len(text_tokens["input_ids"])

        # 拼接 better
        better_full = text + better_response
        better_enc = self.tokenizer(
            better_full,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        # 使用了 apply_chat_template，它会自动在结尾加上 <|assistant|>\n
        # 拼接 worse
        worse_full = text + worse_response
        worse_enc = self.tokenizer(
            worse_full,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )

        return {
            "input_ids_better": better_enc["input_ids"].squeeze(0),
            "attention_mask_better": better_enc["attention_mask"].squeeze(0),
            "input_ids_worse": worse_enc["input_ids"].squeeze(0),
            "attention_mask_worse": worse_enc["attention_mask"].squeeze(0),
            # "text_ids": text_tokens["input_ids"].squeeze(0),
            "text_len": text_len  
        }
train_dataset = PreferenceDataset(tokenizer,split="train",max_length=max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [5]:
'''data set'''

print(train_dataset.dataset[10])
print("the length of the train dataset is:",len(train_dataset.dataset))

{'system': 'You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer.', 'question': 'Q: Answer the following question given this paragraph:   The kidneys also secrete hormones that help maintain homeostasis. For example, they produce a hormone that stimulates bone marrow to produce red blood cells when more are needed. They also secrete a hormone that regulates blood pressure and keeps it in a normal range.   Q: What organs secrete hormones that help maintain homeostasis?   A:\nThe answer is:', 'chosen': 'The kidneys are the organs that secrete hormones to help maintain homeostasis. They produce a hormone that stimulates bone marrow to produce red blood cells when needed, and they also secrete a hormone that regulates blood pressure, keeping it within a normal range.', 'rejected': ' Certainly! Based on the provided paragraph, the organs that secrete hormones to help maintain homeostasis are the kidneys. The kidneys produce two hor

### 设置 Template

### DPO loss
$$
\mathcal{L}_{DPO}(\pi_{\theta};\pi_{ref})=-\mathbb{E}_{(x,y_w,y_l)\sim D}\left[\log \sigma(\beta\log \frac{\pi_{\theta}(y_w | x)}{\pi_{ref}(y_w | x)}-\beta\log \frac{\pi_{\theta}(y_l | x)}{\pi_{ref}(y_l | x)})\right]
$$
**最初的尝试**

首先计算模型输出的 Negative Log-Likelihood：
$$
out\_worse.loss=-\dfrac{1}{N}\sum\limits_{i=1}^N\log P_{\theta}(y_i|x_i)
$$
然后在求 $\log \pi_{\theta}(y_w | x)$ 时使用： $\log\pi_{\theta}(y_l | x)= -out\_worse\_theta.loss$

但是应该采用整个序列预测 token 的对数概率之和，**采用**：
$$
\sum\limits_{i=k+1}^N\log P_{\theta}(y_i|x_{t<i})
$$
其中 k 是不需要计算 loss 的 prompt 输入 token 个数。

In [ ]:
def dpo_loss(logp_theta_w, logp_ref_w, logp_theta_l, logp_ref_l, beta):
    
    diff = beta * ((logp_theta_w - logp_ref_w) - (logp_theta_l - logp_ref_l))
    # 负对数 sigmoid
    loss = -F.logsigmoid(diff).mean()
    return loss

### Train

In [ ]:
def get_log_probs(model, input_ids, attention_mask, labels, is_ref_model=False):
    # 如果是参考模型，强制禁用梯度
    if is_ref_model:
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
    else:
        outputs = model(input_ids, attention_mask=attention_mask)
    
    logits = outputs.logits
    log_probs = F.log_softmax(logits, dim=-1)
    selected_log_probs = log_probs.gather(2, labels.unsqueeze(-1)).squeeze(-1)
    mask = (labels != -100)
    return (selected_log_probs * mask).sum(dim=1)



model.train()
for epoch in range(num_epochs):
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")

    total_loss = 0.0
    total_batches = len(train_dataloader)
    for batch_idx, batch in enumerate(progress_bar):
        

        better_ids   = batch["input_ids_better"].to(device)      # [B, Lp]
        better_att  = batch["attention_mask_better"].to(device)
        worse_ids    = batch["input_ids_worse"].to(device)      # [B, Lc]
        worse_att   = batch["attention_mask_worse"].to(device)
        # prompt_ids   = batch["text_ids"].to(device)    # [B, Lr]
        prompt_len   = batch["text_len"].to(device)

        # make labels
        B, L = better_ids.shape
        w_labels = better_ids.clone()
        token_pos = torch.arange(L, device=device).unsqueeze(0).expand(B, L)
        w_labels[token_pos < prompt_len.unsqueeze(1)] = -100
        l_labels = worse_ids.clone()
        l_labels[token_pos < prompt_len.unsqueeze(1)] = -100

        # 计算策略模型的 log-probs（保留梯度）
        logp_w_theta = get_log_probs(model, better_ids, better_att, w_labels, is_ref_model=False)
        logp_l_theta = get_log_probs(model, worse_ids, worse_att, l_labels, is_ref_model=False)

        # 计算参考模型的 log-probs（禁用梯度）
        logp_w_ref = get_log_probs(ref_model, better_ids, better_att, w_labels, is_ref_model=True)
        logp_l_ref = get_log_probs(ref_model, worse_ids, worse_att, l_labels, is_ref_model=True)

        # ===== DPO loss + backward =====

        loss = dpo_loss(logp_w_theta, logp_w_ref, logp_l_theta, logp_l_ref, beta)
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        total_loss += loss.item()
        avg_loss = total_loss / (batch_idx + 1)

        # 更新进度条显示
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'avg_loss': f'{avg_loss:.4f}'
        })

Epoch 1/1: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 1608/1608 [28:43<00:00,  1.07s/it, loss=0.0014, avg_loss=0.1566]


### Save model

In [8]:
os.makedirs(save_dir, exist_ok=True)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

('./qwen-ft/tokenizer_config.json',
 './qwen-ft/special_tokens_map.json',
 './qwen-ft/vocab.json',
 './qwen-ft/merges.txt',
 './qwen-ft/added_tokens.json',
 './qwen-ft/tokenizer.json')

### Generation

In [9]:
system_msg = "You are an AI assistant that helps people find information."
user_msg   = "Where would the best place to drive over the speed limit be?"

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": user_msg}
]

prompt = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

inputs = tokenizer(prompt, return_tensors="pt").to(device)

model.eval()
with torch.no_grad():
    output_ids = model.generate(**inputs, max_new_tokens=100, do_sample=True)
    ref_output_ids = ref_model.generate(**inputs, max_new_tokens=100, do_sample=False)

output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
ref_output_text = tokenizer.decode(ref_output_ids[0], skip_special_tokens=True)

print("=== Model Output ===")
print(output_text)
print("\n=== Reference Model Output ===")
print(ref_output_text)

/home/miniconda3/envs/align-anything/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/miniconda3/envs/align-anything/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home/miniconda3/envs/align-anything/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


=== Model Output ===
system
You are an AI assistant that helps people find information.
user
Where would the best place to drive over the speed limit be?
assistant


=== Reference Model Output ===
system
You are an AI assistant that helps people find information.
user
Where would the best place to drive over the speed limit be?
assistant
The best place to drive over the speed limit will depend on various factors, including the specific location and circumstances of the driver. However, generally speaking:

1. **Highway**: On highways, it's often safer to drive at a higher speed limit because there is less traffic and more room for maneuvering.

2. **Urban Areas**: In urban areas with limited space, driving at high speeds can lead to accidents due to reduced visibility and increased risk of collisions.

3. **School Zones**: In
